# Creating an ics file from a CSV

### Sanity check to make sure a simple event works

In [2]:
from datetime import datetime
from dateutil import tz

date_format = '%Y-%m-%d %H:%M:%S' # Format the ics module expects the dates to be in
due = '2021-02-01 00:00:00'       # Sample date that needs to be converted to UTC (Currently in Canada/Pacific time zone)

# First let's convert this to a datetime object in the Canada/Pacific Time zone
due_dt = datetime.strptime(due,date_format).replace(tzinfo=tz.gettz('Canada/Pacific'))

# Then let's convert the date to UTC so ics can handle it correctly
due_dt = due_dt.astimezone(tz.tzutc())

# Now let's run the example in the documentation
from ics import Calendar, Event
c = Calendar()
e = Event()
e.name = "My cool event"
e.begin = due_dt.strftime(date_format) # Updated this to use date_format
c.events.add(e)
c.events
# [<Event 'My cool event' begin:2014-01-01 00:00:00 end:2014-01-01 00:00:01>]
with open('my.ics', 'w') as my_file:
    my_file.writelines(c)

##  Start with a template CSV

In [3]:
import pandas as pd

df = pd.read_csv('template_dates.csv')
df

,Assignment_Name,Assignment_DueDate,Comments,Date_TimeZone
0,HW01,2020-08-24 18:00:00,x do this or else,Canada/Pacific
1,HW02,2020-08-31 18:00:00,x do this or else,Canada/Pacific
2,HW03,2020-09-07 18:00:00,x do this or else,Canada/Pacific
3,HW04,2020-09-14 18:00:00,x do this or else,Canada/Pacific
4,HW05,2020-09-21 18:00:00,x do this or else,Canada/Pacific
5,Test01,2020-08-26 18:00:00,x do this or else,Canada/Pacific
6,Test02,2020-09-02 18:00:00,x do this or else,Canada/Pacific
7,Test03,2020-09-09 18:00:00,x do this or else,Canada/Pacific
8,Test04,2020-09-16 18:00:00,x do this or else,Canada/Pacific
9,Test05,2020-09-23 18:00:00,x do this or else,Canada/Pacific


### Make datetime aware with pacific time zone, then convert to UTC (as a new column). 

In [5]:
date_format = '%Y-%m-%d %H:%M:%S' # Format the ics module expects the dates to be in

df['Assignment_DueDate_TZ'] = pd.to_datetime(df['Assignment_DueDate'], 
               format=date_format).apply(lambda x: x.replace(
                                         tzinfo=tz.gettz('Canada/Pacific')).astimezone(tz.tzutc()))

df

,Assignment_Name,Assignment_DueDate,Comments,Date_TimeZone,Assignment_DueDate_TZ
0,HW01,2020-08-24 18:00:00,x do this or else,Canada/Pacific,2020-08-25 01:00:00+00:00
1,HW02,2020-08-31 18:00:00,x do this or else,Canada/Pacific,2020-09-01 01:00:00+00:00
2,HW03,2020-09-07 18:00:00,x do this or else,Canada/Pacific,2020-09-08 01:00:00+00:00
3,HW04,2020-09-14 18:00:00,x do this or else,Canada/Pacific,2020-09-15 01:00:00+00:00
4,HW05,2020-09-21 18:00:00,x do this or else,Canada/Pacific,2020-09-22 01:00:00+00:00
5,Test01,2020-08-26 18:00:00,x do this or else,Canada/Pacific,2020-08-27 01:00:00+00:00
6,Test02,2020-09-02 18:00:00,x do this or else,Canada/Pacific,2020-09-03 01:00:00+00:00
7,Test03,2020-09-09 18:00:00,x do this or else,Canada/Pacific,2020-09-10 01:00:00+00:00
8,Test04,2020-09-16 18:00:00,x do this or else,Canada/Pacific,2020-09-17 01:00:00+00:00
9,Test05,2020-09-23 18:00:00,x do this or else,Canada/Pacific,2020-09-24 01:00:00+00:00


In [6]:
cal = Calendar()

for i,row in df.iterrows():
    e = Event()
    e.name = row['Assignment_Name']
    e.begin = row['Assignment_DueDate_TZ']
    e.description = row['Comments']
    cal.events.add(e)
    
## Export the ics file

with open('template.ics', 'w') as my_file:
    my_file.writelines(cal)